In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import mediapipe as mp
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf



In [2]:
mp_hands = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [7]:
print(len(next(os.walk('Data'))[1]))

60


In [8]:
print(next(os.walk('Data'), (None, None, []))[1])  # [] if no file

['ban dang lam gi', 'ban di dau the', 'ban hieu ngon ngu ky hieu khong', 'ban hoc lop may', 'ban khoe khong', 'ban muon gio roi', 'ban phai canh giac', 'ban ten la gi', 'ban tien bo day', 'ban trong cau co the', 'bo me toi cung la nguoi Diec', 'cai nay bao nhieu tien', 'cai nay la cai gi', 'cam on', 'cap cuu', 'chuc mung', 'chung toi giao tiep voi nhau bang ngon ngu ky hieu', 'con yeu me', 'cong viec cua ban la gi', 'hen gap lai cac ban', 'mon nay khong ngon', 'toi bi chong mat', 'toi bi cuop', 'toi bi dau dau', 'toi bi dau hong', 'toi bi ket xe', 'toi bi lac', 'toi bi phan biet doi xu', 'toi cam thay rat hoi hop', 'toi cam thay rat vui', 'toi can an sang', 'toi can di ve sinh', 'toi can gap bac si', 'toi can phien dich', 'toi can thuoc', 'toi dang an sang', 'toi dang buon', 'toi dang o ben xe', 'toi dang o cong vien', 'toi dang phai cach ly', 'toi dang phan van', 'toi di sieu thi', 'toi di toi Ha Noi', 'toi doc kem', 'toi khoi benh roi', 'toi khong dem theo tien', 'toi khong hieu', 't

In [9]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Data') 

# Actions that we try to detect
actions = np.array(next(os.walk('Data'), (None, None, []))[1])

# Thirty videos worth of data
no_sequences = 60

# Videos are going to be 30 frames in length
sequence_length = 60

In [10]:
print(actions)

['ban dang lam gi' 'ban di dau the' 'ban hieu ngon ngu ky hieu khong'
 'ban hoc lop may' 'ban khoe khong' 'ban muon gio roi'
 'ban phai canh giac' 'ban ten la gi' 'ban tien bo day'
 'ban trong cau co the' 'bo me toi cung la nguoi Diec'
 'cai nay bao nhieu tien' 'cai nay la cai gi' 'cam on' 'cap cuu'
 'chuc mung' 'chung toi giao tiep voi nhau bang ngon ngu ky hieu'
 'con yeu me' 'cong viec cua ban la gi' 'hen gap lai cac ban'
 'mon nay khong ngon' 'toi bi chong mat' 'toi bi cuop' 'toi bi dau dau'
 'toi bi dau hong' 'toi bi ket xe' 'toi bi lac' 'toi bi phan biet doi xu'
 'toi cam thay rat hoi hop' 'toi cam thay rat vui' 'toi can an sang'
 'toi can di ve sinh' 'toi can gap bac si' 'toi can phien dich'
 'toi can thuoc' 'toi dang an sang' 'toi dang buon' 'toi dang o ben xe'
 'toi dang o cong vien' 'toi dang phai cach ly' 'toi dang phan van'
 'toi di sieu thi' 'toi di toi Ha Noi' 'toi doc kem' 'toi khoi benh roi'
 'toi khong dem theo tien' 'toi khong hieu' 'toi khong quan tam'
 'toi la hoc s

In [11]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [46]:
import tensorflow as tf
model = tf.keras.models.load_model('Model/model_CNN_RNN.h5') # main

In [47]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'ban dang lam gi': 0,
 'ban di dau the': 1,
 'ban hieu ngon ngu ky hieu khong': 2,
 'ban hoc lop may': 3,
 'ban khoe khong': 4,
 'ban muon gio roi': 5,
 'ban phai canh giac': 6,
 'ban ten la gi': 7,
 'ban tien bo day': 8,
 'ban trong cau co the': 9,
 'bo me toi cung la nguoi Diec': 10,
 'cai nay bao nhieu tien': 11,
 'cai nay la cai gi': 12,
 'cam on': 13,
 'cap cuu': 14,
 'chuc mung': 15,
 'chung toi giao tiep voi nhau bang ngon ngu ky hieu': 16,
 'con yeu me': 17,
 'cong viec cua ban la gi': 18,
 'hen gap lai cac ban': 19,
 'mon nay khong ngon': 20,
 'toi bi chong mat': 21,
 'toi bi cuop': 22,
 'toi bi dau dau': 23,
 'toi bi dau hong': 24,
 'toi bi ket xe': 25,
 'toi bi lac': 26,
 'toi bi phan biet doi xu': 27,
 'toi cam thay rat hoi hop': 28,
 'toi cam thay rat vui': 29,
 'toi can an sang': 30,
 'toi can di ve sinh': 31,
 'toi can gap bac si': 32,
 'toi can phien dich': 33,
 'toi can thuoc': 34,
 'toi dang an sang': 35,
 'toi dang buon': 36,
 'toi dang o ben xe': 37,
 'toi dang o c

In [54]:
import cv2   
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0) 
# Set mediapipe model 
with mp_hands.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read() # đọc video từ camera

        # Make detections
        image, results = mediapipe_detection(frame, holistic) # gọi biến image và result = giá trị RGB của ảnh
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results) # nhận diện cơ thể 
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results) # xuất ra array các keypoints cơ thể (mấy cái đốt í)
#         sequence.insert(0,keypoints)
        sequence.append(keypoints)
        sequence = sequence[-60:] # Đưa các frame thu đc vào mảng sequence, ở đây t để là 60 frame cuối
        
        if len(sequence) == 60:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)]) # Nếu thu đủ số frame của câu, đưa các array đấy vào model rồi ước lượng 
            print(np.max(res))
            print(res)# ra kết quả chính xác nhất trong dữ liệu
            #print(res[np.argmax(res)])
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 3: 
                sentence = sentence[-3:]

            # Viz probabilities
            # image = prob_viz(res, actions, image, colors)
        
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        if len(sequence) == 60:
            cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 35ms/step
ban tien bo day
0.525391
[2.4560848e-01 9.7020473e-03 1.7465029e-03 6.1798501e-06 8.8290195e-05
 8.0848448e-03 2.0164633e-03 1.6671095e-02 5.2539098e-01 2.2199087e-02
 4.7755469e-07 2.6226130e-03 3.3643465e-03 7.9265219e-06 4.3434419e-08
 1.0142417e-03 3.3553410e-03 5.5056296e-07 6.4879847e-03 2.5391566e-05
 1.3986348e-05 9.9012349e-04 4.6086703e-07 7.6591613e-09 2.5351837e-05
 1.9897178e-03 1.5427311e-05 5.0662975e-03 4.7392566e-03 3.6725447e-07
 1.0207595e-08 2.3849993e-06 1.6807898e-07 5.3163227e-03 3.1487658e-03
 2.4449369e-06 1.5015347e-03 4.3523590e-07 2.7653052e-06 3.5135665e-05
 6.0017005e-04 9.9263176e-05 6.6872798e-07 8.3186014e-06 2.9555749e-05
 1.6922012e-07 6.2860164e-04 1.2518557e-09 1.9093730e-07 8.6997432e-05
 4.2467818e-04 4.4754222e-03 2.6850838e-08 9.8339058e-02 2.3946200e-02
 9.3072500e-05 6.6760575e-09 2.3453133e-06 4.3653517e-06 1.7133245e-05]
<class 'media

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
ban tien bo day
0.6349335
[1.42981783e-01 9.93382931e-03 8.04117415e-03 4.63556034e-06
 9.08513539e-05 2.84211221e-03 6.77609176e-04 9.68160201e-03
 6.34933472e-01 2.34403629e-02 1.64458854e-07 1.62258046e-03
 7.37311842e-04 3.11153116e-07 4.76576378e-09 7.09810806e-03
 1.83985592e-03 1.83355962e-06 8.62393901e-03 1.09867055e-04
 1.23965801e-05 2.88155046e-04 1.24604719e-06 3.01664227e-09
 4.67729624e-05 8.96572892e-04 3.32439299e-06 1.02278823e-02
 1.94449769e-03 4.70356412e-08 4.09642054e-09 3.22158513e-07
 4.22223501e-08 5.41017530e-03 5.73692750e-03 1.01833825e-06
 1.39951101e-03 2.05288018e-08 7.30115516e-07 3.32661307e-06
 1.37864507e-03 1.77904694e-05 1.86304590e-07 1.16022538e-05
 1.66373648e-05 1.02623559e-07 5.40916168e-04 3.61514568e-10
 3.20541034e-07 3.87127038e-05 1.88905338e-04 1.09169427e-02
 6.61570283e-08 8.94859582e-02 1.84991807e-02 2.39181187e-04
 1.86651459

1/1 [==============================] - 0s 30ms/step
cong viec cua ban la gi
0.46356273
[8.67385243e-04 1.30325920e-04 4.29087095e-02 2.01514849e-09
 6.52107656e-06 5.35049821e-07 3.11051990e-04 3.21148150e-02
 1.54925110e-05 9.87017294e-04 4.93871999e-10 3.00617444e-06
 3.06395460e-02 5.86767825e-08 7.13210546e-10 2.10577641e-02
 9.64054198e-04 3.43547290e-05 4.63562727e-01 6.48307521e-03
 1.96885716e-10 8.99678071e-07 7.42826733e-09 1.18448651e-09
 3.32772834e-06 7.55780339e-02 9.10988433e-07 1.98301976e-03
 3.11351869e-07 6.43905834e-04 6.72319018e-08 2.50616124e-13
 4.43247714e-08 6.06966950e-03 1.36650151e-05 1.37915456e-04
 1.81233463e-05 8.40695691e-07 1.08828572e-04 3.65452986e-04
 7.24973828e-02 1.49391475e-04 6.36141704e-05 4.22375779e-05
 8.01719204e-02 4.28567773e-06 1.88985447e-04 6.86853454e-08
 4.65562096e-07 7.34840171e-07 4.00339253e-02 7.74258515e-05
 4.29217744e-06 1.20086037e-01 1.63406180e-03 5.23484971e-07
 6.44695092e-07 1.01241186e-11 6.21914978e-06 2.62116082e-0

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
toi bi ket xe
0.60540354
[2.4867780e-04 1.0796544e-07 5.0756345e-05 5.7883607e-12 2.0744247e-07
 1.1562158e-08 3.1549616e-03 2.7891831e-02 5.5920384e-07 3.1314978e-06
 4.3318246e-13 3.8131642e-10 2.7715715e-02 1.0615306e-06 1.1999110e-10
 9.0626767e-05 3.5989976e-03 8.0160252e-09 1.1514869e-02 2.0070151e-05
 5.8311720e-14 2.4224819e-10 2.0452475e-11 1.6931665e-13 1.9177816e-10
 6.0540354e-01 6.7961605e-09 1.5895157e-05 3.5533077e-08 1.9408796e-03
 5.5631121e-11 2.9958430e-17 7.4236117e-10 6.9460139e-06 9.5756025e-10
 7.8293160e-06 3.6045415e-06 2.8523571e-06 2.2932717e-04 1.9615318e-01
 1.3524516e-03 8.8775664e-04 1.4738351e-05 4.4779631e-07 1.7061712e-02
 1.8725910e-07 1.2599480e-07 7.4681628e-10 5.7977145e-10 1.8356897e-10
 9.9281386e-02 5.2152000e-06 7.2281964e-10 3.3312328e-03 9.0384610e-06
 5.9030048e-10 6.4199457e-10 9.6412321e-17 1.8752768e-09 8.4583940e-09]
<class 'media

...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
toi dang phai cach ly
0.73680776
[5.55972974e-06 1.27174964e-08 9.06231871e-05 4.04607944e-11
 4.14983515e-05 8.43049861e-08 3.19027249e-03 3.98714677e-04
 1.12198286e-07 3.02439162e-07 4.55163129e-10 4.51888349e-09
 1.23085149e-01 9.31836024e-04 4.82520227e-06 4.48641913e-06
 8.01659306e-04 2.12091820e-08 8.46844912e-03 5.11145299e-05
 1.86703647e-12 8.82927065e-09 1.21439067e-10 1.32593558e-09
 5.12276399e-09 1.33209508e-02 6.51309165e-05 3.44947239e-06
 1.60102573e-07 6.26156060e-03 5.78508377e-07 2.81418877e-13
 5.51263474e-06 6.25646615e-04 2.24390782e-08 5.36944601e-04
 5.35651623e-07 9.03807487e-03 1.88928284e-03 7.36807764e-01
 2.79096075e-05 4.84363064e-02 8.31310172e-05 4.37653125e-06
 3.44433114e-02 3.12197117e-05 2.52047329e-07 1.34174377e-06
 4.57079041e-09 2.97013192e-09 9.66427941e-03 1.64359972e-05
 4.05491079e-07 1.63634121e-03 2.43019731e-05 1.34815323e-08
 1.1

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
toi dang o ben xe
0.90044427
[7.83324253e-07 6.17271401e-10 5.48060816e-05 2.06776568e-10
 9.33026429e-03 1.48696813e-07 4.72074002e-03 1.08522718e-05
 1.72466894e-08 1.14099530e-09 5.60118796e-09 1.04328386e-10
 3.00061627e-04 3.00313011e-02 2.34256107e-02 2.84750357e-08
 4.12472803e-03 4.23405933e-09 4.78884764e-03 2.06591722e-06
 7.85032195e-13 1.84510751e-09 2.97115353e-11 4.47502746e-09
 5.95153749e-09 2.90841490e-05 2.46684789e-03 1.06852349e-05
 1.20230334e-06 4.92269057e-04 1.34573028e-08 1.26141675e-12
 4.44590623e-05 1.07536262e-05 7.51048390e-10 1.11525448e-03
 7.81120169e-10 9.00444269e-01 9.56713781e-03 3.43482266e-03
 5.39391287e-08 2.35835614e-05 4.63987902e-07 1.61017218e-04
 1.85363530e-03 2.44815485e-03 1.39339873e-08 6.58403087e-06
 1.21174637e-09 1.31283173e-09 8.52949161e-04 2.29973986e-04
 2.53372747e-07 1.51001313e-05 7.85927128e-08 6.38777294e-07
 5.37893

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
cap cuu
0.53318095
[5.94420044e-07 1.33896116e-10 3.93692972e-05 1.37845924e-09
 2.58982271e-01 1.30649562e-06 1.54455099e-02 1.77732545e-06
 5.64528015e-08 1.01954535e-10 4.85091149e-08 8.68656883e-11
 3.00139868e-06 7.59722590e-02 5.33180952e-01 4.28784963e-09
 3.76021117e-03 1.09668552e-09 2.68188817e-03 7.33578304e-07
 1.00663332e-11 1.09797682e-09 5.24296717e-11 8.82118467e-09
 2.32795436e-08 4.18413251e-08 3.32560437e-03 2.15818945e-05
 4.77809226e-06 9.55635278e-06 9.82256498e-10 3.37336374e-11
 1.51170952e-05 6.72064743e-07 2.27028493e-10 1.81432144e-04
 3.24767366e-11 9.10597965e-02 6.57794997e-03 4.04681577e-06
 2.10870565e-09 3.01301633e-08 5.05266851e-09 3.46444547e-04
 8.94519508e-06 6.44695060e-03 1.42506806e-08 3.20863342e-06
 1.19283194e-09 5.60764191e-09 4.08470369e-05 1.86876860e-03
 2.63302326e-07 1.53380972e-06 4.24101598e-09 1.09149360e-05
 1.27669630e-09 6.

1/1 [==============================] - 0s 31ms/step
ban khoe khong
0.9546757
[1.20810455e-06 4.40964043e-10 8.37188054e-05 6.30850305e-09
 9.54675674e-01 1.43739935e-05 8.54712329e-04 3.72202777e-07
 1.38637859e-07 2.86419721e-10 4.01955816e-07 7.60964486e-11
 3.20844329e-08 1.20798941e-03 1.85356792e-02 6.43134435e-09
 1.20495132e-03 2.79268297e-09 1.93927670e-03 1.35530399e-06
 3.21140163e-11 4.18389412e-09 6.96350477e-11 9.74481917e-09
 1.98950673e-07 2.60358679e-10 5.51448087e-04 1.99423986e-04
 2.60917500e-06 9.86018676e-07 1.09125653e-10 5.58628976e-11
 1.09035477e-06 2.06614914e-07 4.79402074e-10 3.45749359e-05
 4.45788935e-12 1.27721136e-03 7.27031147e-04 1.11934790e-08
 1.78750126e-09 1.60757310e-10 6.60361377e-10 8.33529804e-04
 3.76106698e-07 1.05701182e-02 5.85745070e-08 5.26161557e-07
 1.73463155e-09 2.42993394e-08 6.03862236e-06 7.20771961e-03
 6.61347485e-07 5.20703566e-07 7.09282355e-10 5.94260746e-05
 4.57528931e-10 1.86737403e-09 2.49116715e-06 3.76492017e-06]
<class 

1/1 [==============================] - 0s 24ms/step
ban khoe khong
0.82977486
[6.5151721e-06 1.4618566e-09 7.4895157e-05 2.5631732e-09 8.2977486e-01
 2.9191127e-05 1.3681372e-02 2.1866459e-05 1.2290990e-06 6.6631767e-10
 1.5558194e-07 1.8330756e-09 2.4508231e-06 7.4523039e-02 5.1594391e-02
 1.0982124e-08 9.1165965e-03 1.2366241e-09 7.5100008e-03 3.9173792e-06
 1.4964859e-10 1.5799516e-08 5.6056711e-11 1.4526518e-08 1.7912052e-07
 4.6765138e-09 8.4388466e-04 1.5405069e-04 4.0812688e-06 4.7980438e-06
 2.4566801e-10 6.5402722e-10 2.4689878e-06 7.5916080e-07 2.5106215e-09
 3.1439900e-05 2.8115510e-11 1.6452669e-03 3.7676768e-04 2.1679848e-07
 5.9254961e-09 2.5226023e-09 9.4589481e-10 1.8759923e-04 2.0758785e-06
 2.1892565e-03 5.8746821e-07 3.6391842e-07 8.4395446e-10 6.0823893e-08
 1.0091050e-05 8.1755994e-03 7.9130928e-07 3.3930814e-06 1.1389844e-08
 1.9995627e-05 6.0070166e-10 2.1268431e-09 2.0329551e-06 3.7376899e-06]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [=======